In [ ]:
import pandas as pd
!pip install geopy
from geopy.distance import distance



In [10]:
subway_file_path = "/content/drive/MyDrive/DSL/EDA/서울시 역사마스터 정보_수정.csv"
bus_file_gyeonggi_path = "/content/drive/MyDrive/DSL/EDA/수도권 버스정류장 좌표.csv"
bus_file_seoul_path = "/content/drive/MyDrive/DSL/EDA/서울시버스정류소위치정보(20241209).xlsx"

raw_subway_data = pd.read_csv(subway_file_path)
bus_data_gyeonggi = pd.read_csv(bus_file_gyeonggi_path)
bus_data_seoul = pd.read_excel(bus_file_seoul_path)

subway_cols = ["외부코드", "행정구역", "위도", "경도"]
subway_data = raw_subway_data[subway_cols]

bus_cols = ["정류장번호", "도시명", "위도", "경도"]
bus_data_gyeonggi = bus_data_gyeonggi[bus_cols]

bus_cols_seoul = ["ARS_ID", "Y좌표", "X좌표"]
bus_data_seoul = bus_data_seoul[bus_cols_seoul]
bus_data_seoul.rename(columns={"Y좌표": "위도", "X좌표": "경도"}, inplace=True)

results = []


In [11]:
# 🔄 각 지하철역에 대해 처리
for _, subway_row in subway_data.iterrows():
    subway_id = subway_row["외부코드"]
    subway_lat, subway_lon = subway_row["위도"], subway_row["경도"]
    region = subway_row["행정구역"]

    # 🏙️ 해당 지하철역에 적용할 버스 정류장 필터링
    if region.startswith("서울특별시"):
        filtered_buses = bus_data_seoul  # 서울의 경우, 서울 버스 데이터 사용
    elif region.startswith("경기도") or region.startswith("충청남도"):
        filtered_buses = bus_data_gyeonggi[bus_data_gyeonggi["도시명"] == region]
    elif region.startswith("인천광역시"):
        filtered_buses = bus_data_gyeonggi[bus_data_gyeonggi["도시명"] == "인천광역시"]
    else:
        filtered_buses = pd.DataFrame()  # 기타 지역은 고려하지 않음

    # 🚍 반경 300m 이내 버스 정류장 개수 계산
    count = sum(distance((subway_lat, subway_lon), (bus_row["위도"], bus_row["경도"])).meters <= 300
                for _, bus_row in filtered_buses.iterrows())

    # 결과 저장
    results.append({"외부코드": subway_id, "버스 정류장 개수": count})

In [12]:
result_df = pd.DataFrame(results)
result_df

,외부코드,버스 정류장 개수
0,100-3,5
1,100-2,8
2,100-1,0
3,100,4
4,101,5
...,...,...
797,Y120,4
798,Y121,7
799,Y123,2
800,Y125,10


In [13]:
final_df = subway_data.merge(result_df, on="외부코드", how="left")

final_merged_df = raw_subway_data.merge(final_df[["외부코드", "버스 정류장 개수"]], on="외부코드", how="left")

output_path = "/content/drive/MyDrive/DSL/EDA/서울시_역사마스터_최종.xlsx"
final_merged_df.to_excel(output_path, index=False)

print(f"작업 완료! 결과 파일 저장: {output_path}")

작업 완료! 결과 파일 저장: /content/drive/MyDrive/DSL/EDA/서울시_역사마스터_최종.xlsx
